In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
auctions = pd.read_csv('./datos/auctions.csv')

# A continuación vamos a analizar los tipos de datos del archivo *'auctions'* para poder reducir el tamaño del mismo

Primero que nada, vamos a ver que columnas se tienen y que tipo de dato les asigna Pandas a cada una de ellas.

Podemos ver que el archivo cuenta con **19.571.319 filas** y **7 columnas**, y pesa más de un GigaByte.

In [3]:
auctions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19571319 entries, 0 to 19571318
Data columns (total 7 columns):
auction_type_id    float64
country            int64
date               object
device_id          int64
platform           int64
ref_type_id        int64
source_id          int64
dtypes: float64(1), int64(5), object(1)
memory usage: 1.0+ GB


Luego de haber obtenido un listado de las columnas, vamos a ver cuales de ellas contienen nulos. 

La única que tiene nulos es la columna ***'auction_type_id'***, y tiene una cantidad de 19.571.319. Como podemos ver en la salida anterior, la cantidad de nulos es igual a la cantidad de filas de nuestro archivo, por lo que podemos eliminar dicha columna ya que todos los campos son nulos.

In [ ]:
auctions.isnull().sum()

In [ ]:
auctions = auctions.drop(['auction_type_id'], axis='columns')

In [ ]:
auctions.sample(5)

Analizando la columna ***'country'*** podemos ver que para todos los datos se obtiene el mismo valor y está determinado por el tipo **int64**. Esto nos permite realizar dos acciones depende el enfoque del análisis que haremos con dicho archivo.
+ Si trabajamos solo con este archivo podemos desisir de dicha columna eliminándola para poder reducir el tamaño de los datos. Cuando lleguemos a una conclusión, sabremos de que se tratan de subastas solo para dicho país.
+ Ahora bien, si trabajamos junto con otro archivo, lo que podemos hacer es cambiarle el tipo de dato a ***'category'***. 

En ambas formas, reduciremos bastante el tamaño del archivo.

In [ ]:
auctions['country'].value_counts()

In [ ]:
auctions['country'] = auctions['country'].astype('category')

De una manera bastante parecida podemos tratar a algunas columnas, en ellas no tenemos un solo valor por lo que no es recomendable eliminarlas (salvo que nuestro análisis esté enfocado a otros datos), pero se tienen sólo una cantidad finita (y pequeña) de valores posibles. Por ende, nos servirá cambiar el tipo de **int64** a **category**.
Las columnas son las siguientes: 
+ ***'platform'***
+ ***'ref_type_id'***
+ ***'source_id'***

In [ ]:
auctions['platform'].value_counts()

In [ ]:
auctions['platform'] = auctions['platform'].astype('category')

In [ ]:
auctions['ref_type_id'].value_counts()

In [ ]:
auctions['ref_type_id'] = auctions['ref_type_id'].astype('category')

In [ ]:
auctions['source_id'].value_counts()

In [ ]:
auctions['source_id'] = auctions['source_id'].astype('category')

In [ ]:
auctions.info()

Si nos centramos en la columna ***'date'***, podemos dividir en cuatro columnas más, para poder así separar los datos y entenderlos mejor. Primero lo que debemos hacer es transformar dicha columna al tipo **datetime**.

Después la separaremos en ***'year'***, ***'month'***, ***'day'*** y ***'hour'***.

In [ ]:
auctions['date'] = pd.to_datetime(auctions['date'])

In [ ]:
auctions['year'] = auctions['date'].dt.year
auctions['month'] = auctions['date'].dt.month
auctions['day'] = auctions['date'].dt.day
auctions['hour'] = auctions['date'].dt.hour

In [ ]:
auctions.sample(5)

Si analizamos las columnas ***'year'*** y ***'month'*** podemos ver que cada una solo tiene un valor posible, es decir, que los datos representados en nuetro archivo corresponden al mes de Marzo de 2019. Por lo que podemos eliminar dichas columnas, teniendo en cuenta al final, que el análisis corresponden a dicho período de tiempo.

In [ ]:
auctions['year'].value_counts()

In [ ]:
auctions['month'].value_counts()

In [ ]:
auctions = auctions.drop(['year'], axis='columns')
auctions = auctions.drop(['month'], axis='columns')
auctions = auctions.drop(['date'], axis='columns')

In [ ]:
auctions.info()

Como claramente la hora de las subastas es un número entre 0 y 23, podemos tratar a la columna ***'hour'*** como un **int8** ya que no es necesario representarla a través de un **int64**.
    

In [ ]:
auctions['hour'] = auctions['hour'].astype(np.int8)

In [ ]:
auctions['day'].value_counts()

In [ ]:
auctions['day'] = auctions['day'].astype('category')

In [ ]:
auctions.info()